In [ ]:

# === Montaje de Google Drive ===
from google.colab import drive
drive.mount('/content/drive')

# === Ruta base en Drive (ajústala si es necesario) ===
BASE_DIR = "/content/drive/MyDrive/Facultad/Ciencia de datos/dengue_ckan/AirFlow/include/outputs/"


# === Librerías ===
import os
import sys
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from pandas.api.types import is_numeric_dtype
import matplotlib as mpl
import seaborn as sns


file_path = os.path.join(BASE_DIR, 'dengue_enriched_final.xlsx') # O .xlsx, etc.
df = pd.read_excel(file_path)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# =========================
# PREPROCESAMIENTO DENGUE
# =========================


# 1) Resetear a defaults razonables
sns.reset_defaults()                 # resetea seaborn
mpl.rcParams.update(mpl.rcParamsDefault)  # resetea matplotlib

# -------- utilidades --------
def normalize_text(s: str):
    if pd.isna(s): return s
    s = str(s).strip()
    s = (s
         .replace("Ã‘", "Ñ")
         .replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u")
         .replace("Á","A").replace("É","E").replace("Í","I").replace("Ó","O").replace("Ú","U")
         )
    s = re.sub(r'\s+', ' ', s)
    return s.upper()

def coerce_numeric(x, allow_comma_decimal=True):
    if pd.isna(x): return np.nan
    s = str(x).strip()
    if allow_comma_decimal and (',' in s) and ('.' not in s):
        s = s.replace('.', '')       # separador de miles
        s = s.replace(',', '.')      # coma decimal -> punto
    # quitar posibles miles residuales 1.234 -> 1234 si aplica
    s = re.sub(r'(?<=\d)\.(?=\d{3}\b)', '', s)
    try:
        return float(s)
    except:
        return np.nan

def fix_prov_name(p):
    p = normalize_text(p)
    if p in {"CABA","CIUDAD AUTONOMA BUENOS AIRES","CAPITAL FEDERAL"}:
        return "CIUDAD AUTONOMA DE BUENOS AIRES"
    return p

def standardize_departamento(dep, prov=None):
    dep = normalize_text(dep) if pd.notna(dep) else dep
    prov = fix_prov_name(prov) if pd.notna(prov) else prov
    if prov == "CIUDAD AUTONOMA DE BUENOS AIRES" and isinstance(dep, str):
        m = re.search(r'COMUNA\s*(\d+)', dep)
        if m:
            return f"COMUNA {int(m.group(1))}"
    return dep

# -------- 1) Normalización básica de texto y tipos --------
display(df.head())
if "provincia_nombre" in df.columns:
    df["provincia_nombre"] = df["provincia_nombre"].apply(fix_prov_name)
if "departamento_nombre" in df.columns:
    df["departamento_nombre"] = df.apply(
        lambda r: standardize_departamento(r.get("departamento_nombre"), r.get("provincia_nombre")), axis=1
    )

# Fuerzo numéricos en potenciales columnas climáticas/geo-demográficas
maybe_numeric = [c for c in df.columns if any(k in c.lower() for k in ["lat","lon","temp","hum","prec","poblacion","densidad","superficie"])]
for c in maybe_numeric:
    if c in df.columns and df[c].dtype == "O":
        df[c] = df[c].apply(coerce_numeric)

# # -------- 3) Columna de casos (asegurar existencia y tipo) --------
candidate_case_cols = ["cantidad_casos","casos","n_casos","count_casos"]
case_col = next((c for c in candidate_case_cols if c in df.columns), None)
if case_col is None:
    raise ValueError("No se detectó columna de casos (esperaba una de: cantidad_casos/casos/n_casos/count_casos).")
if df[case_col].dtype == "O":
    df[case_col] = pd.to_numeric(df[case_col].apply(coerce_numeric), errors="coerce")
df[case_col] = df[case_col].fillna(0).clip(lower=0)

# -------- 4) Estandarización robusta de grupo_edad_desc + ID (sin nulos) --------
import re

# Bandas canónicas
_BANDS = [
    (0, 0,  "0 a 0"),
    (1, 4,  "1 a 4"),
    (5, 9,  "5 a 9"),
    (10,14, "10 a 14"),
    (15,19, "15 a 19"),
    (20,24, "20 a 24"),
    (25,34, "25 a 34"),
    (35,44, "35 a 44"),
    (45,64, "45 a 64"),
    (65,200,"65+"),
]

def _age_to_band_inclusive(min_age: int, max_age: int) -> str | None:
    """Devuelve la banda que contiene [min,max]; si cruza, None."""
    if max_age >= 65:
        return "65+"
    for a, b, label in _BANDS:
        if a <= min_age and max_age <= b:
            return label
    return None

def _age_single_band(age: int) -> str:
    """Banda que contiene una edad puntual."""
    if age >= 65:
        return "65+"
    for a, b, label in _BANDS:
        if a <= age <= b:
            return label
    return "DESCONOCIDO"

def _normalize_text_edades(s):
    if pd.isna(s): return None
    s = str(s).strip().lower()
    s = (s.replace("á","a").replace("é","e").replace("í","i")
           .replace("ó","o").replace("ú","u").replace("ñ","n"))
    s = re.sub(r"\s+", " ", s)
    return s

def standardize_grupo_edad(desc) -> str:
    """
    Siempre devuelve una etiqueta canónica: '0 a 0', '1 a 4', ..., '65+' o 'DESCONOCIDO'.
    Política cuando cruza bandas: usar la BANDA DEL MÁXIMO (conservadora).
    """
    s = _normalize_text_edades(desc)
    if s is None or s in {"", "sin especificar", "sin esp"}:
        return "DESCONOCIDO"

    # Número puro -> banda
    if re.fullmatch(r"\d{1,3}", s):
        return _age_single_band(int(s))

    # Casos especiales
    if "neonato" in s or "posneonato" in s or ("menor" in s and "1" in s):
        return "0 a 0"
    if "igual a 1" in s:
        return "1 a 4"  # unificamos a la banda canónica

    # Rangos "de X a Y" / "X a Y" / "X hasta Y"
    m = re.search(r"(\d+)\s*(?:de\s+)?(?:a|hasta)\s*(\d+)", s)
    if m:
        x, y = int(m.group(1)), int(m.group(2))
        lo, hi = (x, y) if x <= y else (y, x)
        lab = _age_to_band_inclusive(lo, hi)
        if lab:
            return lab
        # Cruza bandas -> usar banda del máximo (conservadora)
        return _age_single_band(hi)

    # 65+ variantes
    if ("mayor" in s and "65" in s) or "65 y mas" in s or "65+" in s:
        return "65+"

    # Varios números sueltos (p.ej., "1 2 3 4 5 7")
    nums = [int(n) for n in re.findall(r"\d+", s)]
    if len(nums) >= 1:
        # Política: usar banda del máximo
        return _age_single_band(max(nums))

    return "DESCONOCIDO"

# Aplicar al DF
if "grupo_edad_desc" in df.columns:
    df["grupo_edad_desc_std"] = df["grupo_edad_desc"].apply(standardize_grupo_edad)
else:
    df["grupo_edad_desc_std"] = "DESCONOCIDO"

# Mapeo a ID (incluye DESCONOCIDO=99 para evitar nulos)
ID_MAP = {
    "0 a 0": 0,
    "1 a 4": 1,
    "5 a 9": 2,
    "10 a 14": 3,
    "15 a 19": 4,
    "20 a 24": 5,
    "25 a 34": 6,
    "35 a 44": 7,
    "45 a 64": 8,
    "65+": 9,
    "DESCONOCIDO": 99
}
df["grupo_edad_id"] = df["grupo_edad_desc_std"].map(ID_MAP).astype("Int64")


# -------- 5) Completar IDs de provincia y departamento --------
def completar_id_por_modo(df, nombre_col, id_col, adicionales_keys=None):
    if nombre_col not in df.columns or id_col not in df.columns:
        return df
    d = df.copy()
    # Mapa nombre -> id más frecuente observado
    mapa = (d[[nombre_col, id_col]]
            .dropna()
            .groupby(nombre_col)[id_col]
            .agg(lambda s: s.value_counts().idxmax())
            .to_dict())
    # Completar faltantes
    mask = d[id_col].isna() & d[nombre_col].notna()
    d.loc[mask, id_col] = d.loc[mask, nombre_col].map(mapa)

    # (opcional) completar con códigos deterministas si aún faltan
    if d[id_col].isna().any():
        # CABA: reglas típicas
        if nombre_col == "provincia_nombre":
            d.loc[d[id_col].isna() & (d[nombre_col] == "CIUDAD AUTONOMA DE BUENOS AIRES"), id_col] = 2
        if nombre_col == "departamento_nombre":
            if "provincia_nombre" in d.columns:
                m = d.loc[d[id_col].isna() & (d["provincia_nombre"]=="CIUDAD AUTONOMA DE BUENOS AIRES"), nombre_col].str.extract(r'COMUNA\s*(\d+)')
                idxs = m.dropna().index
                d.loc[idxs, id_col] = 2000 + m.loc[idxs, 0].astype(int)

        # Para el resto, generar IDs deterministas por hash corto
        still = d[id_col].isna() & d[nombre_col].notna()
        if still.any():
            base = d.loc[still, nombre_col].astype(str)
            if adicionales_keys and all(k in d.columns for k in adicionales_keys):
                extra = d.loc[still, adicionales_keys].astype(str).agg("|".join, axis=1)
                key = (base + "|" + extra)
            else:
                key = base
            d.loc[still, id_col] = key.map(lambda s: abs(hash(s)) % 10_000_000 + 1_000_000)
            d[id_col] = pd.to_numeric(d[id_col], errors="coerce").astype("Int64")
    return d

# Provincia
if "provincia_nombre" in df.columns and "provincia_id" in df.columns:
    df = completar_id_por_modo(df, "provincia_nombre", "provincia_id")
elif "provincia_nombre" in df.columns:
    df["provincia_id"] = pd.Series([pd.NA]*len(df), dtype="Int64")
    df = completar_id_por_modo(df, "provincia_nombre", "provincia_id")

# Departamento (usa provincia como llave adicional)
if "departamento_nombre" in df.columns and "departamento_id" in df.columns:
    df = completar_id_por_modo(df, "departamento_nombre", "departamento_id", adicionales_keys=["provincia_nombre"])
elif "departamento_nombre" in df.columns:
    df["departamento_id"] = pd.Series([pd.NA]*len(df), dtype="Int64")
    df = completar_id_por_modo(df, "departamento_nombre", "departamento_id", adicionales_keys=["provincia_nombre"])

# -------- 6) Fecha semanal (si aún no existe) --------
from datetime import date
def iso_week_start_safe(year, week):
    try: return pd.to_datetime(date.fromisocalendar(int(year), int(week), 1))
    except: return pd.NaT

if "fecha_semana" not in df.columns:
    if {"anio","semana_epidemiologica"}.issubset(df.columns):
        df["fecha_semana"] = df.apply(lambda r: iso_week_start_safe(r["anio"], r["semana_epidemiologica"]), axis=1)
    elif "fecha" in df.columns:
        df["fecha_semana"] = pd.to_datetime(df["fecha"], errors="coerce")
    else:
        df["fecha_semana"] = pd.NaT
df = df[df["fecha_semana"].notna()].copy()

# -------- 7) Promedio semanal de clima (precipitación, temperatura, humedad) --------
climate_cols = [c for c in df.columns if any(k in c.lower() for k in ["temp","hum","prec"]) and is_numeric_dtype(df[c])]
if not climate_cols:
    print("⚠️ No se detectaron columnas climáticas numéricas.")
else:
    dias = ["_L","_M","_X","_J","_V","_S","_D"]
    def promedio_intrafila(df, base):
        cols = [c for c in df.columns if c.lower().startswith(base) and any(c.endswith(d) for d in dias)]
        if cols:
            return df[cols].mean(axis=1)
        return None

    for base in ["temp", "hum", "prec"]:
        col_prom = f"{base}_sem_prom"
        val = promedio_intrafila(df, base)
        if isinstance(val, pd.Series):
            df[col_prom] = val

    for base in ["temp","hum","prec"]:
        if f"{base}_sem_prom" not in df.columns:
            cand = [c for c in climate_cols if c.lower().startswith(base)]
            if cand:
                df[f"{base}_row"] = df[cand].mean(axis=1)

    group_keys = [k for k in ["fecha_semana","provincia_nombre","departamento_nombre"] if k in df.columns]
    if not group_keys:
        group_keys = ["fecha_semana"]

    agg_dict = {case_col: "sum"}
    for base in ["temp","hum","prec"]:
        if f"{base}_sem_prom" in df.columns:
            agg_dict[f"{base}_sem_prom"] = "mean"
        if f"{base}_row" in df.columns:
            agg_dict[f"{base}_row"] = "mean"

    clima_sem = df.groupby(group_keys).agg(agg_dict).reset_index()

    clima_sem = clima_sem.rename(columns={
        "temp_sem_prom": "temp_semana",
        "hum_sem_prom": "hum_semana",
        "prec_sem_prom": "prec_semana",
        "temp_row": "temp_semana",
        "hum_row": "hum_semana",
        "prec_row": "prec_semana",
    })

    clima_cols_finales = [c for c in ["temp_semana","hum_semana","prec_semana"] if c in clima_sem.columns]
    if clima_cols_finales:
        clima_sem = (clima_sem
                     .groupby(group_keys, as_index=False)
                     .agg({case_col:"sum", **{c:"mean" for c in clima_cols_finales}}))

    print("clima_sem (muestra):")
    display(clima_sem.head())


# -------- 7.1) Columnas mes/año desde 'fecha' y filtro enero–junio --------
# Asegurar que 'fecha' exista y sea datetime
if "fecha" in df.columns:
    df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")
elif "fecha_semana" in df.columns:
    df["fecha"] = pd.to_datetime(df["fecha_semana"], errors="coerce")
else:
    raise ValueError("No se encontró 'fecha' ni 'fecha_semana' para derivar mes/año.")

# Crear columnas nuevas desde 'fecha'
df["mes"] = df["fecha"].dt.month.astype("Int64")
df["anio_fecha"] = df["fecha"].dt.year.astype("Int64")

# Filtrar meses de enero (1) a junio (6) inclusive
df = df[df["mes"].between(1, 6)].copy()

# (opcional) si también querés filtrar la tabla agregada 'clima_sem' cuando la generes,
# asegurate de que 'clima_sem' tenga una columna de fecha (o derivala igual que arriba)
# y aplicá el mismo filtro. Por ejemplo, si usás 'fecha_semana' en clima_sem:
# if "clima_sem" in locals():
#     if "fecha_semana" in clima_sem.columns:
#         clima_sem["mes"] = pd.to_datetime(clima_sem["fecha_semana"]).dt.month
#         clima_sem = clima_sem[clima_sem["mes"].between(1,6)].copy()


# -------- 9) Reporte final de nulos --------
print("\n--- VERIFICACIÓN DE NULOS (post-procesamiento) ---")
display(df.isna().sum().sort_values(ascending=False).to_frame("n_nulos"))
print("Filas x Columnas:", df.shape)


# --- Diagnóstico: revisar cuántos quedaron como DESCONOCIDO ---
n_total = len(df)
n_descon = (df["grupo_edad_desc_std"] == "DESCONOCIDO").sum()
porc = (n_descon / n_total * 100) if n_total > 0 else 0

print(f"Registros con grupo_edad_desc_std = 'DESCONOCIDO': {n_descon:,} de {n_total:,} ({porc:.2f}%)")

# Ver ejemplos de los valores originales que generaron 'DESCONOCIDO'
display(
    df.loc[df["grupo_edad_desc_std"] == "DESCONOCIDO", ["grupo_edad_desc"]]
      .value_counts()
      .reset_index(name="frecuencia")
      .head(20)
)

df_nulos = df[df["grupo_edad_desc_std"].isna()]
display(df_nulos)


In [ ]:
# =========================
# Clasificación climática por provincia -> df["clima_region"]
# Cobertura 100% (sin MIXTO/OTROS) + aviso de provincias no mapeadas
# =========================

def fix_prov_name(p):
    if pd.isna(p):
        return p
    p = str(p).strip().upper()
    if p in {"CABA","CIUDAD AUTONOMA BUENOS AIRES","CAPITAL FEDERAL","CIUDAD AUTONOMA DE BUENOS AIRES"}:
        return "CABA"
    # tildes básicas
    repl = str.maketrans("ÁÉÍÓÚÑ", "AEIOUN")
    return p.translate(repl)

# Normalizá provincia
if "provincia_nombre" in df.columns:
    df["provincia_nombre"] = df["provincia_nombre"].apply(fix_prov_name)
else:
    raise ValueError("Falta la columna 'provincia_nombre' en df.")

# --- Mapeo exhaustivo por provincia (24 jurisdicciones) ---
# Criterio:
#   - TEMPLADO: Buenos Aires, CABA, Entre Ríos, Santa Fe, Córdoba, La Pampa
#   - SUBTROPICAL: Misiones, Chaco, Corrientes, Formosa
#   - ARIDO/SEMIARIDO: Catamarca, La Rioja, San Juan, San Luis, Santiago del Estero, Santa Cruz, Tierra del Fuego...
#   - FRIO/MONTANA: Mendoza, Neuquén, Río Negro, Chubut, Jujuy, Salta
PROVINCIA_A_CLIMA = {
    # TEMPLADO
    "BUENOS AIRES": "TEMPLADO",
    "CIUDAD AUTONOMA DE BUENOS AIRES": "TEMPLADO",
    "CABA": "TEMPLADO",
    "ENTRE RIOS": "TEMPLADO",
    "SANTA FE": "TEMPLADO",
    "CORDOBA": "TEMPLADO",
    "LA PAMPA": "TEMPLADO",

    # SUBTROPICAL (NEA)
    "MISIONES": "SUBTROPICAL",
    "CHACO": "SUBTROPICAL",
    "CORRIENTES": "SUBTROPICAL",
    "FORMOSA": "SUBTROPICAL",
    "TUCUMAN": "SUBTROPICAL",

    # ARIDO/SEMIARIDO (Puna/Sierras/Patagonia extraandina)
    "CATAMARCA": "ARIDO/SEMIARIDO",
    "LA RIOJA": "ARIDO/SEMIARIDO",
    "SAN JUAN": "ARIDO/SEMIARIDO",
    "SAN LUIS": "ARIDO/SEMIARIDO",
    "SANTIAGO DEL ESTERO": "ARIDO/SEMIARIDO",
    "SANTA CRUZ": "ARIDO/SEMIARIDO",
    "TIERRA DEL FUEGO, ANTARTIDA E ISLAS DEL ATLANTICO SUR": "ARIDO/SEMIARIDO",
    "TIERRA DEL FUEGO": "ARIDO/SEMIARIDO",

    # FRIO/MONTANA (Cordillera/NOA andino + Norpatagonia andina)
    "MENDOZA": "FRIO/MONTANA",
    "NEUQUEN": "FRIO/MONTANA",
    "RIO NEGRO": "FRIO/MONTANA",
    "CHUBUT": "FRIO/MONTANA",
    "JUJUY": "FRIO/MONTANA",
    "SALTA": "FRIO/MONTANA",
}

# Asignación primaria por mapeo
df["clima_region"] = df["provincia_nombre"].map(PROVINCIA_A_CLIMA)

# Detectar provincias no cubiertas por el mapeo
provincias_en_df = set(df["provincia_nombre"].dropna().unique())
provincias_mapeadas = set(PROVINCIA_A_CLIMA.keys())
faltantes = sorted(p for p in provincias_en_df if p not in provincias_mapeadas)

if faltantes:
    print("⚠️ Provincias no reconocidas en el mapeo y asignadas por DEFAULT a 'TEMPLADO':")
    for p in faltantes:
        print("  -", p)
    # Fallback operativo para no cortar el análisis
    df.loc[df["provincia_nombre"].isin(faltantes), "clima_region"] = "TEMPLADO"

# Chequeo final: garantizar sin nulos
if df["clima_region"].isna().any():
    # Si aún quedara algún nulo (p.ej. provincia NaN), forzar a TEMPLADO
    df["clima_region"] = df["clima_region"].fillna("TEMPLADO")




df_grouped = df.groupby(
    ["provincia_nombre", "departamento_nombre", "anio", "semana_epidemiologica","fecha_semana","temp_sem_prom","hum_sem_prom","prec_sem_prom","clima_region","densidad","grupo_edad_id","grupo_edad_desc","grupo_edad_desc_std" ]
)["cantidad_casos"].sum().reset_index()

In [ ]:
import altair as alt
import pandas as pd
import numpy as np
import uuid

alt.renderers.enable('default')
alt.data_transformers.enable('default', max_rows=None)

# ==== datos (ajusta si hace falta) ====
reg_col = "clima_region"
features = ["densidad", "temp_sem_prom", "hum_sem_prom", "prec_sem_prom"]
use_cols = ["provincia_nombre","departamento_nombre","fecha_semana", reg_col] + features
dfv = (df_grouped[use_cols]
       .dropna(subset=[reg_col] + features)
       .rename(columns={reg_col: "region"})
       .copy())
dfv["fecha_semana"] = pd.to_datetime(dfv["fecha_semana"], errors="coerce")
meses_map = {1:"Enero",2:"Febrero",3:"Marzo",4:"Abril",5:"Mayo",6:"Junio",
             7:"Julio",8:"Agosto",9:"Septiembre",10:"Octubre",11:"Noviembre",12:"Diciembre"}
dfv["mes_desc"] = dfv["fecha_semana"].dt.month.map(meses_map)

# ==== opciones de dropdown ====
regiones = sorted(dfv["region"].dropna().unique().tolist())
meses_desc = [meses_map[m] for m in sorted(dfv["fecha_semana"].dt.month.dropna().unique())]

# ==== parámetros (NO son selections, no generan *_tuple) ====
SUF = str(uuid.uuid4())[:6]  # por si querés evitar colisiones entre celdas
param_region = alt.param(name=f"region_sel_{SUF}",
                         bind=alt.binding_select(options=["(todas)"] + regiones, name="Región: "))
param_mes    = alt.param(name=f"mes_sel_{SUF}",
                         bind=alt.binding_select(options=["(todos)"] + meses_desc, name="Mes: "))

# ==== filtros usando los params (dejan pasar todo si no se elige nada o se elige '(todas)/(todos)') ====
filtro_region = f"""!isValid({param_region.name}) || {param_region.name}=="" ||
                   {param_region.name}=="(todas)" || datum.region=={param_region.name}"""
filtro_mes    = f"""!isValid({param_mes.name})    || {param_mes.name}=="" ||
                   {param_mes.name}=="(todos)"   || datum.mes_desc=={param_mes.name}"""

# ======= EJEMPLO DE DOS GRÁFICOS QUE COMPARTEN LOS MISMOS PARAMS =======

# 1) Heatmap “huella” (z-scores por región y variable)
mean_by_reg_feat = (dfv.melt(id_vars=["region"], value_vars=features,
                             var_name="variable", value_name="valor")
                      .groupby(["region","variable"], as_index=False)["valor"].mean())
z_df = []
for v in features:
    sub = mean_by_reg_feat[mean_by_reg_feat["variable"] == v].copy()
    mu, sd = sub["valor"].mean(), sub["valor"].std(ddof=0)
    sub["z"] = (sub["valor"] - mu) / (sd if sd > 0 else 1.0)
    z_df.append(sub)
z_df = pd.concat(z_df, ignore_index=True)
order_vars = (z_df.groupby("variable")["z"].apply(lambda s: s.max()-s.min())
                  .sort_values(ascending=False).index.tolist())

heat = (alt.Chart(z_df)
          .transform_filter(filtro_region)  # usa el param, pero NO lo agrega acá
          .mark_rect()
          .encode(
              x=alt.X("variable:N", title="Variable", sort=order_vars),
              y=alt.Y("region:N",   title="Región"),
              color=alt.Color("z:Q", title="Z-score",
                              scale=alt.Scale(scheme="blueorange", domainMid=0)),
              tooltip=["region:N","variable:N",alt.Tooltip("valor:Q",format=".2f"),alt.Tooltip("z:Q",format=".2f")]
          )
          .properties(title="Huella por región (z-score por variable)", width=420, height=160))

# 2) Boxplots facetados con filtros por región y mes (mismo param)
long_df = dfv.melt(id_vars=["region","provincia_nombre","departamento_nombre","mes_desc"],
                   value_vars=features, var_name="variable", value_name="valor").dropna(subset=["valor"])
box = (alt.Chart(long_df)
         .transform_filter(filtro_region)
         .transform_filter(filtro_mes)
         .mark_boxplot(outliers=True)
         .encode(
             y=alt.Y("region:N", title="Región", sort=regiones),
             x=alt.X("valor:Q",  title="Valor"),
             color=alt.Color("region:N", legend=None),
             tooltip=["region:N","variable:N","valor:Q"]
         )
         .properties(width=250, height=120))
box_grid = (box.facet(column=alt.Column("variable:N", title=None, sort=order_vars))
                 .resolve_scale(x="independent")
                 .properties(title="Distribución de variables por región (filtrá opcionalmente)"))

# 👉 Agregá los params SOLO en el contenedor
layout = (heat | box_grid).add_params(param_region, param_mes)
layout
